In [1]:
import os

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.sequential import SequentialChain
from langchain.callbacks import get_openai_callback
import json
import pandas as pd
import traceback
from dotenv import load_dotenv   
import PyPDF2

In [3]:
load_dotenv()

True

In [4]:
open_key = os.getenv('openai_key')

In [5]:
llm = ChatOpenAI(openai_api_key = open_key, model_name='gpt-3.5-turbo',temperature =0.7)

f:\ML\NLP\MCQ-Generator\venv\Lib\site-packages\langchain_core-0.1.52-py3.11.egg\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
llm

In [10]:
template = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}
"""

In [11]:
prompt_template = PromptTemplate(
    input_variables=['text','number','subject','tone','RESPONSE_JSON'],
    template=template
)

In [14]:
with open('F:/ML/NLP/MCQ-Generator/Response.json','r') as f:
    RESPONSE_JSON = json.load(f)

In [15]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [16]:
quiz_chain = LLMChain(llm =llm, prompt=prompt_template, output_key = 'quiz',verbose=True)

f:\ML\NLP\MCQ-Generator\venv\Lib\site-packages\langchain_core-0.1.52-py3.11.egg\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [17]:
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [18]:
evalution_prompt_template = PromptTemplate(
    input_variables=['subject','quiz'],
    template=template2
)

In [23]:
review_chain = LLMChain(llm =llm, prompt=evalution_prompt_template, output_key = 'review',verbose=True)

In [24]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','RESPONSE_JSON'],output_variables=['quiz','review'])

In [26]:
# paragraph for MCQ generation
with open('F:/ML/NLP/MCQ-Generator/data.txt') as f:
    text = f.read()

In [27]:
text

'Cricket is a bat-and-ball game that is played between two teams of eleven players on a field at the centre of which is a 22-yard (20-metre) pitch with a wicket at each end, each comprising two bails balanced on three stumps. Two players from the batting team (the striker and nonstriker) stand in front of either wicket, with one player from the fielding team (the bowler) bowling the ball towards the striker\'s wicket from the opposite end of the pitch. The striker\'s goal is to hit the bowled ball and then switch places with the nonstriker, with the batting team scoring one run for each exchange. Runs are also scored when the ball reaches or crosses the boundary of the field or when the ball is bowled illegally.\n\nThe fielding team tries to prevent runs from being scored by dismissing batters (so they are "out"). Means of dismissal include being bowled, when the ball hits the striker\'s wicket and dislodges the bails, and by the fielding side either catching the ball after it is hit b

In [ ]:
number = 5
subject = 'Sport'
tone = 'Simple'

In [ ]:

with get_openai_callback() as cb:
    generate_evaluate_chain(
        'text':text,
        'number':number,
        'subject':subject,
        'tone':tone,
        'RESPONSE_JSON':RESPONSE_JSON
    )